#  Стохастические градиентные методы

## Постановка задачи

$$
\min_x \frac1N \sum_{i=1}^N f_i(x)
$$

- Целевая функция - сумма конечного числа функций
- $N$ может быть очень большим

### Особенности задачи

- Точное вычисление градиента занимает очень много времени
- Высокая точность решения обычно не требуется
- Допустимо введение случайности

## Использование структуры функции

- Так как целевая функция равна сумме функций, то градиент равен сумме градиентов
- **Основная идея:** вместо сложения всех градиентов, сложить только некоторые, выбранные некотором образом
- Как выбирать?

## Стохастический градиентный спуск (SGD)

$$
x_{k+1} = x_k - \alpha_k \sum_{i \in \mathcal{I}} f'_i(x_k)
$$

Сходимость

- Для выпуклых функций $\mathcal{O}(1 / \sqrt{k})$
- Для сильно выпуклых функций $\mathcal{O}(1 / k)$

## Ускоренный градиентный метод

$$
\begin{align*}
& h_k = \beta h_{k-1} - \alpha_k \sum_{i\in \mathcal{I}} f'_i(x_k + \alpha_k h_k)\\
& x_{k+1} = x_k + h_k
\end{align*}
$$

- Не приводит к ускорению в теории!
- Почему?
- Различные реализации в различных пакетах могут называться одинаково

## Adagrad

\begin{align*}
& g_k = \sum_{i \in \mathcal{I}} f'_i(x_k)\\
& r = r + g_k \cdot g_k\\
& h_k = -\frac{\varepsilon}{\delta + \sqrt{r}} \cdot g_k\\
& x_{k+1} = x_k + h_k
\end{align*}

- Диагональное шкалирование градиента
- Учёт **всей** предыдущей истории для вычисления следующей точки
- Не улучшает теоретические оценки сходимости

## Adam

\begin{align*}
& g_k = \sum_{i \in \mathcal{I}} f'_i(x_k)\\
& r = \rho_2r + (1 - \rho_2) g_k \cdot g_k\\
& s = \rho_1 s + (1 - \rho_1) g_k\\
& \hat{s} = \frac{s}{1 - \rho_1^k}\\
& \hat{r} = \frac{r}{1 - \rho_2^k}\\
& h_k = -\frac{\varepsilon \hat{s}}{\delta + \sqrt{\hat{r}}} \\
& x_{k+1} = x_k + h_k
\end{align*}

- Все операции выполняются поэлементно

## Другие модификации

<img src="optax_common.png" width=400>

- Список взят из документации к [optax](https://optax.readthedocs.io/en/latest/)
- Как вы думаете, что можно улучшать в рассмотренных выше методах?

## Основные направления улучшений

- Настройка размера шага
- Использование дополнительного шума для избегания локальных минимумов
- Сокращение дополнительной памяти

## Выводы

- Использование знания о структуре функции позволяет ускорить процесс вычисления градиента
- За это приходится платить снижением точности
- Адаптивные размеры шага генерируются на основании истории вычисленных градиентов
- Иногда адекватно работают и для невыпуклых функций
- Ускоренные методы работают на практике гораздо лучше, чем в теории